In [1]:
import os
import gc
import sys
import random
import datetime
import importlib
import itertools
import numpy as np
from scipy import spatial
import scipy.sparse as sparse
import scipy.stats as stats
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns

from mayavi import mlab

# os.chdir('/media/sina/Windows1/Users/smansourlako/Documents/Reserach/Codes/fMRI')
os.chdir('/home/sina/Documents/Research/Codes/fMRI')

# import constants as cs
import myconstants as cs
# import importlib.util
# spec = importlib.util.spec_from_file_location('cs', '/home/sina/Documents/Research/Codes/fMRI/constants.py')
# cs = importlib.util.module_from_spec(spec)
# sys.modules['cs'] = cs
# spec.loader.exec_module(cs)
import utils
import niutils
import hcp


---

Let's first load all the files needed:

In [2]:
from nibabel import freesurfer

# load the pial and white surfaces for left and right hemispheres

# each loaded object is a python tuple containing two elements:
#     1. the coordinates of all vertices
#     2. the triangle information of the mesh

lh_pial_surf = freesurfer.read_geometry('/home/sina/Documents/Research/Datasets/UK_biobank/sample/1000094/FreeSurfer/surf/lh.pial')
lh_white_surf = freesurfer.read_geometry('/home/sina/Documents/Research/Datasets/UK_biobank/sample/1000094/FreeSurfer/surf/lh.white')

rh_pial_surf = freesurfer.read_geometry('/home/sina/Documents/Research/Datasets/UK_biobank/sample/1000094/FreeSurfer/surf/rh.pial')
rh_white_surf = freesurfer.read_geometry('/home/sina/Documents/Research/Datasets/UK_biobank/sample/1000094/FreeSurfer/surf/rh.white')



In [83]:
# coordinates
(lh_pial_surf[0].shape, rh_pial_surf[0].shape, lh_white_surf[0].shape, rh_white_surf[0].shape)

((127142, 3), (131099, 3), (127142, 3), (131099, 3))

In [16]:
# triangles
lh_pial_surf[1].shape

(262194, 3)

In [28]:
# Now let's load the surface atlas mapped to each surface

# each loaded object is a python tuple containing 3 elements:
#     1. A list of integers indicating the label of each vertex
#     2. A table for the color of each label (r, g, b, t, colortable array id)
#     3. A list names of the lables


# lh_glasser_annot = freesurfer.read_annot('/home/sina/Documents/Research/Datasets/UK_biobank/sample/1000094/FreeSurfer/label/lh.native.glasser.annot')
# rh_glasser_annot = freesurfer.read_annot('/home/sina/Documents/Research/Datasets/UK_biobank/sample/1000094/FreeSurfer/label/rh.native.glasser.annot')

lh_glasser_annot = freesurfer.read_annot('/home/sina/Documents/Research/Codes/UKB-connectomics/data/temporary/subjects/test_sub/atlases/lh.native.Schaefer7n500p.annot')
rh_glasser_annot = freesurfer.read_annot('/home/sina/Documents/Research/Codes/UKB-connectomics/data/temporary/subjects/test_sub/atlases/rh.native.Schaefer7n500p.annot')



In [4]:
lh_glasser_annot[0]

array([184, 184, 184, ..., 249, 249, 249], dtype=int32)

In [30]:
np.unique(np.concatenate([lh_glasser_annot[0], rh_glasser_annot[0]]))

array([  0,   1,   2,   3,   4,   6,   7,   9,  11,  12,  14,  15,  17,
        18,  19,  20,  23,  26,  33,  34,  35,  36,  37,  38,  39,  40,
        42,  44,  46,  47,  50,  54,  55,  57,  58,  59,  62,  63,  67,
        68,  69,  73,  74,  75,  77,  78,  79,  81,  83,  85,  87,  88,
        89,  90,  91,  92,  93,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 106, 107, 109, 110, 111, 113, 114, 115, 116, 117, 118, 119,
       120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
       134, 135, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 155, 156, 157, 158, 159, 160, 162, 163, 164, 165, 166, 167,
       168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180,
       181, 182, 183, 184, 185, 188, 189, 190, 192, 193, 200, 201, 202,
       203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 214, 215, 216,
       217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229,
       230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 24

In [29]:
len(np.unique(np.concatenate([lh_glasser_annot[0], rh_glasser_annot[0]])))

393

In [85]:
(lh_glasser_annot[0].shape, rh_glasser_annot[0].shape)

((127142,), (131099,))

In [25]:
lh_glasser_annot[1]

array([[     255,      255,      255,      255, 16777215],
       [      69,        0,      255,        0, 16711749],
       [      49,      109,      132,        0,  8678705],
       ...,
       [     116,       48,       46,        0,  3027060],
       [     133,       69,       90,        0,  5916037],
       [     122,       37,       73,        0,  4793722]], dtype=int32)

In [32]:
lh_glasser_annot[2][:6]

[b'???', b'R_V1_ROI', b'R_MST_ROI', b'R_V6_ROI', b'R_V2_ROI', b'R_V3_ROI']

In [4]:
# Finally we need to load the ribbon mask to use as a reference for voxels to be labeled (only label voxels in the cortical ribbon)

# The ribbon file can be read using nibabel. The saved object is an MGHImage object containing the affine matrix, as well as the labels in a 3d data matrix
# Check https://nipy.org/nibabel/reference/nibabel.freesurfer.html#nibabel.freesurfer.mghformat.MGHImage

# Label numbers are from the FreeSurferColorLUT (https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/AnatomicalROI/FreeSurferColorLUT) lookup table and has 5 values:
#     0: Unknown
#     2: Left-Cerebral-White-Matter
#     3: Left-Cerebral-Cortex
#     41: Right-Cerebral-White-Matter
#     42: Right-Cerebral-Cortex

ribbon = nib.load('/home/sina/Documents/Research/Datasets/UK_biobank/sample/1000094/FreeSurfer/mri/ribbon.mgz')


In [36]:
ribbon

In [38]:
ribbon.affine

array([[-1.00000000e+00,  0.00000000e+00,  1.16415322e-10,
         1.28081192e+02],
       [ 1.16415322e-10,  2.98023224e-08,  1.00000012e+00,
        -1.66473007e+02],
       [ 2.32830644e-10, -1.00000012e+00,  0.00000000e+00,
         6.58933563e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [177]:
# Note: the vox2ras_tkr is different from the affine!
# Question: which one should be used? -- needs to be tested to be verified
ribbon.header.get_vox2ras_tkr()

array([[  -1.,    0.,    0.,  128.],
       [   0.,    0.,    1., -128.],
       [   0.,   -1.,    0.,  128.],
       [   0.,    0.,    0.,    1.]], dtype=float32)

In [185]:
# Note: the vox2ras_tkr is different from the affine!
# Question: which one should be used? -- needs to be tested to be verified
ribbon.header.get_vox2ras()

array([[-1.0000000e+00,  0.0000000e+00,  1.1641532e-10,  1.2808119e+02],
       [ 1.1641532e-10,  2.9802322e-08,  1.0000001e+00, -1.6647301e+02],
       [ 2.3283064e-10, -1.0000001e+00,  0.0000000e+00,  6.5893356e+01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]],
      dtype=float32)

In [186]:
# Note: the vox2ras_tkr is different from the affine!
# Question: which one should be used? -- needs to be tested to be verified
ribbon.header.get_ras2vox()

array([[-1.00000000e+00,  1.16415308e-10,  3.46944612e-18,
         1.28081192e+02],
       [-2.32830616e-10,  2.71050478e-20, -9.99999881e-01,
         6.58933487e+01],
       [ 1.16415315e-10,  9.99999881e-01,  2.98023153e-08,
         1.66472992e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]], dtype=float32)

In [51]:
ribbon.get_fdata().shape

(256, 256, 256)

In [50]:
np.unique(ribbon.get_fdata())

array([ 0.,  2.,  3., 41., 42.])

In [5]:
id_to_label = {
    0: 'Unknown',
    2: 'Left-Cerebral-White-Matter',
    3: 'Left-Cerebral-Cortex',
    41: 'Right-Cerebral-White-Matter',
    42: 'Right-Cerebral-Cortex',
}

label_id = {id_to_label[x]:x for x in id_to_label}


In [7]:
label_id

{'Unknown': 0,
 'Left-Cerebral-White-Matter': 2,
 'Left-Cerebral-Cortex': 3,
 'Right-Cerebral-White-Matter': 41,
 'Right-Cerebral-Cortex': 42}

In [75]:
[id_to_label[x] for x in np.unique(ribbon.get_fdata())]

['Unknown',
 'Left-Cerebral-White-Matter',
 'Left-Cerebral-Cortex',
 'Right-Cerebral-White-Matter',
 'Right-Cerebral-Cortex']

In [7]:
# Finally we need to load the ribbon mask to use as a reference for voxels to be labeled (only label voxels in the cortical ribbon)

# The ribbon file can be read using nibabel. The saved object is an MGHImage object containing the affine matrix, as well as the labels in a 3d data matrix
# Check https://nipy.org/nibabel/reference/nibabel.freesurfer.html#nibabel.freesurfer.mghformat.MGHImage

# Label numbers are from the FreeSurferColorLUT (https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/AnatomicalROI/FreeSurferColorLUT) lookup table and has 5 values:
#     0: Unknown
#     2: Left-Cerebral-White-Matter
#     3: Left-Cerebral-Cortex
#     41: Right-Cerebral-White-Matter
#     42: Right-Cerebral-Cortex

ribbon = nib.load('/home/sina/Documents/Research/Datasets/UK_biobank/sample/1000094/FreeSurfer/mri/ribbon.mgz')

# Given the potential problems with mgz files, mainly the overflow of uint8, I tried converting the original ngz to nii
# the following command was used:
# mri_convert --in_type mgz --out_type nii --out_orientation RAS FreeSurfer/mri/ribbon.mgz FreeSurfer/mri/ribbon.nii.gz

# ribbonnii = nib.load('/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/ribbon.nii.gz')


In [8]:
ribbon

---

Now we can generate the new volumetric labels:

In [9]:
# We'll use kdtree's to store the coordinates in data structure to query for nearest neighbor

# One kdtree per hemisphere

lh_pial_xyz = lh_pial_surf[0]
lh_white_xyz = lh_white_surf[0]
lh_xyz = np.vstack([lh_pial_xyz, lh_white_xyz])
# lh_white_kdtree = spatial.KDTree(lh_white_xyz)
# lh_pial_kdtree = spatial.KDTree(lh_pial_xyz)
lh_kdtree = spatial.KDTree(lh_xyz)

rh_pial_xyz = rh_pial_surf[0]
rh_white_xyz = rh_white_surf[0]
rh_xyz = np.vstack([rh_pial_xyz, rh_white_xyz])
# rh_white_kdtree = spatial.KDTree(rh_white_xyz)
# rh_pial_kdtree = spatial.KDTree(rh_pial_xyz)
rh_kdtree = spatial.KDTree(rh_xyz)


In [173]:
np.min(lh_xyz, axis=0)

array([-71.22108459, -82.33670807, -45.23505402])

In [172]:
np.min(lh_cortex_xyz, axis=0)

array([ -70.91880797, -120.47299741, -107.10666426])

In [10]:
# create a copy of the ribbon label file to overwrite
glasser_labels = (ribbon.get_fdata().copy()*0).astype(float)


In [11]:
# extract the indices of voxels in the cortical ribbon

lh_cortex_ijk = np.array(np.where(ribbon.get_fdata()==label_id['Left-Cerebral-Cortex'])).T

rh_cortex_ijk = np.array(np.where(ribbon.get_fdata()==label_id['Right-Cerebral-Cortex'])).T


In [12]:
# use the affine transformation to get to xyz coordinates of the voxels

# lh_cortex_xyz = nib.affines.apply_affine(ribbon.affine, lh_cortex_ijk)
lh_cortex_xyz = nib.affines.apply_affine(ribbon.header.get_vox2ras_tkr(), lh_cortex_ijk)
# lh_cortex_xyz = nib.affines.apply_affine(ribbon.header.get_ras2vox(), lh_cortex_ijk)

# rh_cortex_xyz = nib.affines.apply_affine(ribbon.affine, rh_cortex_ijk)
rh_cortex_xyz = nib.affines.apply_affine(ribbon.header.get_vox2ras_tkr(), rh_cortex_ijk)
# rh_cortex_xyz = nib.affines.apply_affine(ribbon.header.get_ras2vox(), rh_cortex_ijk)


In [13]:
# querry each voxel's coordinates to find the nearest neighbor on the surfaces (takes a few seconds)

# lh_distance, lh_index = lh_white_kdtree.query(lh_cortex_xyz)
# lh_distance, lh_index = lh_pial_kdtree.query(lh_cortex_xyz)
lh_distance, lh_index = lh_kdtree.query(lh_cortex_xyz)

# rh_distance, rh_index = rh_white_kdtree.query(rh_cortex_xyz)
# rh_distance, rh_index = rh_pial_kdtree.query(rh_cortex_xyz)
rh_distance, rh_index = rh_kdtree.query(rh_cortex_xyz)


In [14]:
# convert the indices to a surface index (reduce the white matter and pial indices to one)

lh_index_surf = lh_index%(lh_glasser_annot[0].shape[0])

rh_index_surf = rh_index%(rh_glasser_annot[0].shape[0])


In [15]:
# write appropriate labels from glasser atlas to volume

glasser_labels[ribbon.get_fdata()==label_id['Left-Cerebral-Cortex']] = lh_glasser_annot[0][lh_index_surf]
# glasser_labels[ribbon.get_fdata()==label_id['Left-Cerebral-Cortex']] = 181

glasser_labels[ribbon.get_fdata()==label_id['Right-Cerebral-Cortex']] = rh_glasser_annot[0][rh_index_surf]
# glasser_labels[ribbon.get_fdata()==label_id['Right-Cerebral-Cortex']] = 10


In [18]:
(np.unique(glasser_labels))

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144., 145., 146., 147., 148., 149., 15

In [17]:
len(np.unique(glasser_labels))

359

In [29]:
ribbonnii.header.get_data_dtype()

dtype('uint8')

In [30]:
# now write the label into an mgh freesurfer volumetric format

# 
# writing mgz files didn't work due to uint8 overflow
# 
# freesurfer.mghformat.MGHImage(
#     glasser_labels.astype(np.uint), # dataobj
#     ribbon.affine, # affine
# ).to_file_map({
#     'image': nib.fileholders.FileHolder('/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasserwtt.mgz')
# })
# 

img = nib.nifti1.Nifti1Image(
    glasser_labels,
#     ribbonnii.affine,
    ribbon.header.get_vox2ras(),
#     header=ribbonnii.header,
)

nib.save(
    img,
    '/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasser.nii.gz'
)


---

Let's also create a color lookup table in freesurfer format:

In [302]:
len(np.arange(len(lh_glasser_annot[2])))

361

In [299]:
np.savetxt(
    '/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasserColorLUT.txt',
    np.array(
        pd.DataFrame({
            '#No.': np.arange(len(lh_glasser_annot[2])),
            'Label Name': [x.decode('utf8') for x in lh_glasser_annot[2]],
            'R': lh_glasser_annot[1][:, 0],
            'G': lh_glasser_annot[1][:, 1],
            'B': lh_glasser_annot[1][:, 2],
            'A': lh_glasser_annot[1][:, 3],
        })
    ),
    fmt=['%d','%s','%d','%d','%d','%d',]
)

---

Let's also create a color lookup table to be used with the connectome workbench viewer:

**Note**: The mgz files are converted to nifti using the following command:

`mri_convert --in_type mgz --out_type nii --out_orientation RAS FreeSurfer/mri/brain.mgz FreeSurfer/mri/brain.nii.gz`

Then, the [volume-label-import](https://www.humanconnectome.org/software/workbench-command/-volume-label-import) is used to convert the glasser.nii.gz into a dlabel file which can be openned in workbench viewer:

`wb_command -volume-label-import '/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasser.nii.gz' '/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasser_label_list.txt' '/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasser_wb_label.nii.gz'`

The label list file is created bellow:

In [307]:
atlas_labels = [x.decode('utf8') for x in lh_glasser_annot[2]]

with open('/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasser_label_list.txt', 'w') as label_list_file:
    for i in range(1,len(lh_glasser_annot[2])):
        label_list_file.write(f'{atlas_labels[i]}\n{i} {lh_glasser_annot[1][i, 0]} {lh_glasser_annot[1][i, 1]} {lh_glasser_annot[1][i, 2]} {255-lh_glasser_annot[1][i, 3]}\n')
        

---

Resample the volumetric label to fMRI space:

Freesurfer's mri_vol2vol seems to work fine:

`mri_vol2vol --mov '/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasser.nii.gz' --targ '/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/fMRI/rfMRI.ica/mean_func.nii.gz' --interp nearest --regheader --o '/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasser_resampled_to_fMRI.nii.gz'`

Followed by the following to create workbench labels:

`wb_command -volume-label-import '/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasser_resampled_to_fMRI.nii.gz' '/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasser_label_list.txt' '/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasser_resampled_to_fMRI_wb_label.nii.gz'`


In [321]:
# not helpful as generates interpolations

# from nibabel.processing import resample_from_to

# fMRI_mean = nib.load('/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/fMRI/rfMRI.ica/mean_func.nii.gz')

# resampled_img = resample_from_to(img, to_vox_map=fMRI_mean, mode='nearest')

# nib.save(resampled_img, '/home/sina/Documents/Research/Datasets/UK biobank/sample/1000094/FreeSurfer/mri/glasser_resampled_nib.nii.gz')


---

Let's try some visualization: